In [ ]:
!nvidia-smi

In [ ]:
import os
import torch

# 1. Clone Repository (Safe to re-run)
if not os.path.exists("InfiniteTalk"):
    !git clone https://github.com/MeiGen-AI/InfiniteTalk
else:
    print("Repository already cloned.")

# 2. Enter Directory (Required after every restart)
%cd InfiniteTalk

# 3. Safe Dependency Installation (Uses Pre-installed Torch)
print(f"✅ Detected System PyTorch Version: {torch.__version__}")
print("Installing compatible extensions... (Skipping massive re-install to prevent crashes)")

# Install Flash Attention (Attempt to find compatible build)
!pip install --no-cache-dir packaging ninja
!pip install --no-cache-dir flash_attn --no-build-isolation

# Install other dependencies from requirements.txt
# We modify requirements to remove rigid constraints that break Colab
!sed -i 's/numpy>=1.23.5,<2/numpy>=1.23.5/' requirements.txt
!pip install --no-cache-dir -r requirements.txt
!pip install --no-cache-dir librosa huggingface_hub

# Install system dependencies
!apt-get update && apt-get install -y ffmpeg > /dev/null 2>&1

print("✅ Setup Complete! You likely DO NOT need to restart the runtime. Try running the next cell.")

In [ ]:
import os
os.makedirs("weights", exist_ok=True)

# Download base model and InfiniteTalk weights
!hf download Wan-AI/Wan2.1-I2V-14B-480P --local-dir ./weights/Wan2.1-I2V-14B-480P
!hf download MeiGen-AI/InfiniteTalk --local-dir ./weights/InfiniteTalk

# Download English audio encoder (DEFAULT)
!hf download facebook/wav2vec2-base-960h --local-dir ./weights/wav2vec2-base-960h

# Alternative: For Chinese audio support, uncomment the lines below:
# !hf download TencentGameMate/chinese-wav2vec2-base --local-dir ./weights/chinese-wav2vec2-base
# !hf download TencentGameMate/chinese-wav2vec2-base model.safetensors --revision refs/pr/1 --local-dir ./weights/chinese-wav2vec2-base

In [ ]:
# Enable public link for Gradio
!sed -i 's/demo.launch(server_name="0.0.0.0", debug=True, server_port=8418)/demo.launch(server_name="0.0.0.0", debug=True, server_port=8418, share=True)/' app.py

# Launch Gradio interface with English audio encoder (DEFAULT)
!python app.py --ckpt_dir weights/Wan2.1-I2V-14B-480P --wav2vec_dir 'weights/wav2vec2-base-960h' --infinitetalk_dir weights/InfiniteTalk/single/infinitetalk.safetensors --num_persistent_param_in_dit 0 --motion_frame 9

# Alternative: For Chinese audio, uncomment below and comment the English line above:
# !python app.py --ckpt_dir weights/Wan2.1-I2V-14B-480P --wav2vec_dir 'weights/chinese-wav2vec2-base' --infinitetalk_dir weights/InfiniteTalk/single/infinitetalk.safetensors --num_persistent_param_in_dit 0 --motion_frame 9